In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


基本変数定義

In [2]:
sampling_flg=0 #サンプリング有無をコントロール
submit_flg=1 #保存するかをコントロール（サンプリングしない時のみ）

SEED=12345
sample_num=10000
fold_num=5

#train関連
train_dir='../../01_input/train.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='../../01_input/test.csv'
test_drop_col=['ID_code']

#結果ファイル関連　nameは自分の名前に変更する
train_preds_dir='../../03_predict_train/name_200_LightGBM_train.csv'
test_preds_dir='../../04_predict_test/name_200_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='../../01_input/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

前処理

In [3]:
#ロード
train_df=pd.read_csv(train_dir)
test_df=pd.read_csv(test_dir)

In [4]:
#サンプリング
if sampling_flg ==1:
    train_df=train_df.sample(n=sample_num,random_state=SEED)
    test_df=test_df.sample(n=sample_num,random_state=SEED)

In [5]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

モデル実行

In [6]:
#model_param
param = {
    "objective" : "binary", 
    "boost":"gbdt",
    "metric":"auc",
    "boost_from_average":"false",
    "num_threads":28,
    "learning_rate" : 0.001,
    "num_leaves" : 13,
    "max_depth":-1,
    "tree_learner" : "serial",
    "feature_fraction" : 0.05,
    "bagging_freq" : 5,
    "bagging_fraction" : 0.4,
    "min_data_in_leaf" : 80,
    "min_sum_hessian_in_leaf" : 10.0,
    "verbosity" : 1,
    'seed': 44000,
    }

In [ ]:
folds = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(x_train), 1))
test_preds = np.zeros((len(x_test), 1))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train.values, y_train.values)):
    print("\n")
    print("Fold {}".format(fold_))
    trn_x,trn_y = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x,val_y = x_train.iloc[val_idx], y_train.iloc[val_idx]
    trn_data = lgb.Dataset(trn_x,trn_y)
    val_data = lgb.Dataset(val_x,val_y)
    
    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    val_pred = clf.predict(val_x, num_iteration=clf.best_iteration)
    test_pred = clf.predict(x_test, num_iteration=clf.best_iteration)
    
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    #print("val = {}".format(val_pred)
    oof_preds[val_idx, :] = val_pred.reshape((-1, 1))
    test_preds += test_pred.reshape((-1, 1))
    
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))
    
test_preds /= fold_num
roc_score = roc_auc_score(y_train, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))
print(confusion_matrix(y_train, pd.DataFrame(np.round(oof_preds))))



Fold 0
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.890405	valid_1's auc: 0.8745
[2000]	training's auc: 0.891541	valid_1's auc: 0.875849
[3000]	training's auc: 0.893871	valid_1's auc: 0.877656
[4000]	training's auc: 0.895792	valid_1's auc: 0.879174
[5000]	training's auc: 0.897186	valid_1's auc: 0.88035
[6000]	training's auc: 0.898601	valid_1's auc: 0.881512
[7000]	training's auc: 0.899954	valid_1's auc: 0.882457
[8000]	training's auc: 0.901208	valid_1's auc: 0.883416
[9000]	training's auc: 0.902484	valid_1's auc: 0.884302
[10000]	training's auc: 0.903824	valid_1's auc: 0.885188
[11000]	training's auc: 0.905011	valid_1's auc: 0.885969
[12000]	training's auc: 0.906189	valid_1's auc: 0.886756
[13000]	training's auc: 0.907399	valid_1's auc: 0.887604
[14000]	training's auc: 0.908496	valid_1's auc: 0.888294
[15000]	training's auc: 0.909578	valid_1's auc: 0.888919
[16000]	training's auc: 0.910642	valid_1's auc: 0.889595
[17000]	training's auc: 0.9

[46000]	training's auc: 0.931418	valid_1's auc: 0.897893
[47000]	training's auc: 0.931915	valid_1's auc: 0.898006
[48000]	training's auc: 0.932421	valid_1's auc: 0.898125
[49000]	training's auc: 0.932911	valid_1's auc: 0.898243
[50000]	training's auc: 0.933394	valid_1's auc: 0.898335
[51000]	training's auc: 0.93387	valid_1's auc: 0.898424
[52000]	training's auc: 0.934354	valid_1's auc: 0.89849
[53000]	training's auc: 0.934831	valid_1's auc: 0.898577
[54000]	training's auc: 0.935297	valid_1's auc: 0.898663
[55000]	training's auc: 0.935763	valid_1's auc: 0.898726
[56000]	training's auc: 0.93622	valid_1's auc: 0.898776
[57000]	training's auc: 0.936677	valid_1's auc: 0.898835
[58000]	training's auc: 0.937123	valid_1's auc: 0.898907
[59000]	training's auc: 0.937563	valid_1's auc: 0.898959
[60000]	training's auc: 0.938002	valid_1's auc: 0.899002
[61000]	training's auc: 0.938436	valid_1's auc: 0.89904
[62000]	training's auc: 0.938869	valid_1's auc: 0.899077
[63000]	training's auc: 0.939306	va

In [ ]:
#結果保存
if (submit_flg ==1 and sampling_flg==0):
    series_oof_preds = pd.Series(data=oof_preds[:,0], name=save_col_name, dtype='float')
    series_oof_preds.to_csv(train_preds_dir,header=True, index=False)

    sample = pd.read_csv(sample_submission_dir)
    sample.target = test_preds[:,0].astype(float)
    sample.ID_code = test_df['ID_code']
    sample.to_csv(test_preds_dir, index=False)
    